In [1]:
import findspark
findspark.init()

import pyspark
from pyspark import SparkContext

In [2]:
sc=SparkContext(master="local[4]")

In [3]:
logs = sc.textFile("input_files/*.csv")

In [4]:
logs.count()

40000

In [5]:
logs.distinct().count()

40000

In [6]:
logs.take(1)

['eefc2eb34a354ab7acaf14ae784a37c9,2019-09-12T00:00:25.920Z,http://example.com/?url=169,user-041,CR,IE,Android,201,0.2856']

In [118]:
#read through lines, keep only the dateHour and url
def getDateHourUrl(line):
    columns = line.split(',')
    dateHour = columns[1].split(':')[0]
    url = columns[2]
    return ( dateHour, [dateHour, url] )

In [119]:
#get all the dateHour
def getDateHour(line):
    columns = line.split(',')
    dateHour = columns[1].split(':')[0]
    return ( dateHour )

In [126]:
dateHourUrlCollection = logs.map(getDateHourUrl)

In [124]:
dateHourUrlCollection.take(2)

[('2019-09-12T00', ['2019-09-12T00', 'http://example.com/?url=169']),
 ('2019-09-12T00', ['2019-09-12T00', 'http://example.com/?url=183'])]

In [100]:
dateHourCollection = logs.map(getDateHour)

In [101]:
dateHourUrlCollection.distinct().count()

13417

In [102]:
dateHourCollection.distinct().count()

73

In [105]:
dateHourUrlCountList = []
for dateHour in dateHourCollection.distinct().collect():
    count = dateHourUrlCollection.filter(lambda x: x[0] == dateHour).distinct().count()
    dateHourUrlCountList.append([dateHour, count])

In [107]:
sorted( dateHourUrlCountList )

[['2019-09-12T00', 183],
 ['2019-09-12T01', 188],
 ['2019-09-12T02', 186],
 ['2019-09-12T03', 191],
 ['2019-09-12T04', 184],
 ['2019-09-12T05', 182],
 ['2019-09-12T06', 189],
 ['2019-09-12T07', 185],
 ['2019-09-12T08', 188],
 ['2019-09-12T09', 190],
 ['2019-09-12T10', 188],
 ['2019-09-12T11', 188],
 ['2019-09-12T12', 186],
 ['2019-09-12T13', 185],
 ['2019-09-12T14', 186],
 ['2019-09-12T15', 185],
 ['2019-09-12T16', 190],
 ['2019-09-12T17', 189],
 ['2019-09-12T18', 189],
 ['2019-09-12T19', 191],
 ['2019-09-12T20', 189],
 ['2019-09-12T21', 183],
 ['2019-09-12T22', 186],
 ['2019-09-12T23', 184],
 ['2019-09-13T00', 189],
 ['2019-09-13T01', 183],
 ['2019-09-13T02', 186],
 ['2019-09-13T03', 189],
 ['2019-09-13T04', 190],
 ['2019-09-13T05', 181],
 ['2019-09-13T06', 184],
 ['2019-09-13T07', 189],
 ['2019-09-13T08', 177],
 ['2019-09-13T09', 190],
 ['2019-09-13T10', 187],
 ['2019-09-13T11', 189],
 ['2019-09-13T12', 190],
 ['2019-09-13T13', 187],
 ['2019-09-13T14', 186],
 ['2019-09-13T15', 186],


In [139]:
f = open('spark_queries/spark_q1.csv', 'w')
for item in sorted( dateHourUrlCountList ):
    f.write(   "%s,%s\n" % (item[0], str(item[1]))   )

In [ ]:
dateHourUrlCollection.map(lambda x: (x[0], x[1]))\
    .groupByKey()\
    .mapValues(lambda vals: len(set(vals)))\
    .sortByKey()\
    .collect()